In [2]:
import pandas as pd
import numpy as np
from LX2_masterscan import mz_ml_paths, spectra_2_df


[11:40:28] {comtypes.client._code_cache:95} INFO - Imported existing <module 'comtypes.gen' from 'c:\\Users\\mirandaa\\Anaconda3\\envs\\lx128_p3_Dev\\lib\\site-packages\\comtypes\\gen\\__init__.py'>
[11:40:28] {comtypes.client._code_cache:72} INFO - Using writeable comtypes cache directory: 'c:\Users\mirandaa\Anaconda3\envs\lx128_p3_Dev\lib\site-packages\comtypes\gen'


In [13]:
import pickle
with open('optoins.pkl','rb') as f: options = pickle.load(f)

In [ ]:
mzmls = mz_ml_paths(options)
samples = [p.stem for p in mzmls]

In [4]:
spectra_dfs = spectra_2_df(options)


[11:40:51] {ms_deisotope.data_source.scan.loader:376} INFO - Disposing of MzMLLoader('test_resources\\small_test\\190321_Serum_Lipidextract_368723_01.mzML') with 2 extant scans attached to it.
[11:40:52] {LX2_masterscan:304} INFO - spectra 190321_Serum_Lipidextract_368723_01, size: (368085, 8)
[11:41:07] {ms_deisotope.data_source.scan.loader:376} INFO - Disposing of MzMLLoader('test_resources\\small_test\\190321_Serum_Lipidextract_368723_02.mzML') with 2 extant scans attached to it.
[11:41:07] {LX2_masterscan:304} INFO - spectra 190321_Serum_Lipidextract_368723_02, size: (389793, 8)


In [5]:
for spectra_df in spectra_dfs:
    break

In [ ]:
def guess_selection_window (spectra_df):
    precursor_mzs = spectra_df.loc[~spectra_df.precursor_id.isna()].precursor_mz.drop_duplicates()
    precursor_mzs.sort_values(inplace=True)
    agg_data = precursor_mzs.diff().agg(['mean','std'])

    mean_less_std = agg_data.at['mean'] - agg_data.at['std']
    if mean_less_std > 0:
        res =  mean_less_std / 2
    else:
        res =  agg_data.at['mean'] /2

    return res

In [ ]:
# suggest tolerance
# sugest calibration masses using suggested tolerance
scan_count = spectra_df.loc[spectra_df.precursor_id.isna()].scan_id.unique().size


In [ ]:
scan_count

In [18]:
df = spectra_df.loc[spectra_df.precursor_id.isna()]

In [19]:
df.sort_values('mz', inplace=True)

c:\Users\mirandaa\Anaconda3\envs\lx128_p3_Dev\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [8]:
def bin_linear_alignment(masses, options):
    # TODO assert masses are ordered
    up_to = None
    for _, mass in masses.iteritems():
        if up_to is None:
            #up_to = mass + tolerance.getTinDA(mass) this is how its done in some places, but reuslt are not identical to below
            up_to = mass + mass / options["MSresolution"].tolerance 
        if mass <= up_to:
            yield up_to
        else:
            up_to = mass + mass / options["MSresolution"].tolerance
            yield up_to


In [20]:
bins1 = list(bin_linear_alignment(df.mz, options))
bins2 = list(bin_linear_alignment(df.groupby(bins1)["mz"].transform("mean"), options))
bins3 = list(bin_linear_alignment(df.groupby(bins2)["mz"].transform("mean"),options))

df['bin_mass'] = bins3 

c:\Users\mirandaa\Anaconda3\envs\lx128_p3_Dev\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


------------------------------------

-------------------------------------

In [21]:
df.sort_values('mz', ascending = False, inplace=True)
# repetition_rate = 0.7
scan_count = df.scan_id.unique().size
df['mz_diff'] = df.mz.diff(-1)
df['mz_diff_long'] = df['mz_diff'].rolling(scan_count).mean() # neg because of sorting order
# short_count = scan_count * repetition_rate
# df['mz_diff_short'] = df.mz.diff().rolling(int(short_count)).mean()
# df['scan_id_f'] = df['scan_id'].factorize()[0]
# df['scan_cnt'] = df.scan_id_f.rolling(scan_count).apply(lambda s: len(set(s)))



c:\Users\mirandaa\Anaconda3\envs\lx128_p3_Dev\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\Users\mirandaa\Anaconda3\envs\lx128_p3_Dev\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
c:\Users\mirandaa\Anaconda3\envs\lx128_p3_Dev\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [22]:

def diff_bin(df):

    cur_bin = 0
    curr_long = 0.01

    for tup in df.itertuples():
        yield cur_bin
        if tup.mz_diff > curr_long:
            cur_bin +=1
        
        if tup.mz_diff_long < curr_long:
            curr_long = tup.mz_diff_long



In [ ]:
df.to_clipboard()

In [23]:
df['diff_bin'] = list(diff_bin(df))

c:\Users\mirandaa\Anaconda3\envs\lx128_p3_Dev\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
def get_collapsable_bins(df):
    df['scan_id_f'] = df['scan_id'].factorize()[0]
    grouped = df.groupby('diff_bin')
    grouped_stats = grouped.agg({'mz':['max','min','std']})
    close_mz = grouped_stats[('mz','min')] - grouped_stats[('mz','max')].shift(-1) < grouped_stats[('mz','std')] + grouped_stats[('mz','std')].shift(-1)
    close_mz_groups = close_mz[close_mz | close_mz.shift(1)].index.to_numpy()
    close_sets = df.loc[df.diff_bin.isin(close_mz_groups)].groupby('diff_bin')['scan_id_f'].apply(lambda s:set(s))

    collapsable_map = {}
    prev_set = set()
    prev_idx = 0
    merging = False
    for idx, curr_set in close_sets.iteritems():
        if (idx - prev_idx <=1 or merging) and not curr_set.intersection(prev_set):
            collapsable_map[idx] = prev_idx
            prev_set.update(curr_set)
            merging = True
        prev_idx = idx
        prev_set = curr_set
        merging = False
    
    return collapsable_map

In [79]:
df['scan_id_f'] = df['scan_id'].factorize()[0]
grouped = df.groupby('diff_bin')
grouped_stats = grouped.agg({'mz':['max','min','std']})
close_mz = grouped_stats[('mz','min')] - grouped_stats[('mz','max')].shift(-1) < grouped_stats[('mz','std')] + grouped_stats[('mz','std')].shift(-1)
close_mz_groups = close_mz[close_mz | close_mz.shift(1)].index.to_numpy()
close_sets = df.loc[df.diff_bin.isin(close_mz_groups)].groupby('diff_bin')['scan_id_f'].apply(lambda s:set(s))

c:\Users\mirandaa\Anaconda3\envs\lx128_p3_Dev\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [80]:

res = {}
prev_set = set()
prev_idx = 0
merging = False
for idx, curr_set in close_sets.iteritems():
    if (idx - prev_idx <=1 or merging) and not curr_set.intersection(prev_set):
        res[idx] = prev_idx
        prev_set.update(curr_set)
        merging = True
    prev_idx = idx
    prev_set = curr_set
    merging = False

In [81]:
res

{74: 73,
 119: 118,
 169: 168,
 356: 355,
 389: 388,
 393: 392,
 438: 437,
 468: 467,
 481: 480,
 503: 502,
 567: 566,
 577: 576,
 585: 584,
 601: 600,
 622: 621,
 635: 634,
 638: 637,
 642: 641,
 654: 653,
 655: 654,
 682: 681,
 692: 691,
 694: 693,
 716: 715,
 718: 717,
 747: 746,
 749: 748,
 788: 787,
 806: 805,
 844: 843,
 861: 860,
 872: 871,
 897: 896,
 901: 900,
 908: 907,
 909: 908,
 910: 909,
 945: 944,
 950: 949,
 1014: 1013,
 1029: 1028,
 1030: 1029,
 1045: 1044,
 1047: 1046,
 1072: 1071,
 1089: 1088,
 1098: 1097,
 1110: 1109,
 1117: 1116,
 1131: 1130,
 1137: 1136,
 1197: 1196,
 1221: 1220,
 1239: 1238,
 1241: 1240,
 1259: 1258,
 1286: 1285,
 1300: 1299,
 1311: 1310,
 1343: 1342,
 1355: 1354,
 1405: 1404,
 1413: 1412,
 1434: 1433,
 1472: 1471,
 1489: 1488,
 1494: 1493,
 1506: 1505,
 1513: 1512,
 1515: 1514,
 1524: 1523,
 1544: 1543,
 1581: 1580,
 1589: 1588,
 1603: 1602,
 1612: 1611,
 1627: 1626,
 1715: 1714,
 1728: 1727,
 1817: 1816,
 1821: 1820,
 1850: 1849,
 1851: 1850,
 

In [ ]:
# for merge collapse
df['scan_id_f'] = df['scan_id'].factorize()[0]
sets = df.groupby('diff_bin')['scan_id_f'].apply(lambda s:set(s))

In [ ]:
def prev_intersetct(sets):
    prev = set()
    for curr in sets:
        yield(curr.intersection(prev))
        prev = curr

In [ ]:
ints = list(prev_intersetct(sets))

In [ ]:
res = []

prev_scan_id_set = ()
prev_min = 100_000
prev_std = 0
prev_cluster = 0
for idx, gdf in df.groupby('diff_bin'):
    if prev_min - gdf.mz.max() < prev_std + gdf.mz.std():
        curr_set = set(gdf.scan_id_f)
        if not curr_set.intersection(prev_scan_id_set):
            prev_scan_id_set.update(curr_set)
            res.append((idx,prev_cluster))
        #candidarte
    else:
        prev_cluster = idx
        prev_std = gdf.mz.std()
        prev_min = gdf.mz.min()
        prev_scan_id_set = set(gdf.scan_id_f)


In [ ]:
res

--------------------


In [ ]:
df['scan_cnt'] = df.scan_id_f.rolling(scan_count).apply(lambda s: len(set(s)))

In [ ]:
df['scan_id_f'] = df['scan_id'].factorize()[0]

In [ ]:
df.sort_values(['mz','scan_id_f'], ascending=False, inplace=True)
df

In [ ]:
df['mz_diff_31'] = df.mz.diff(-31)
df['mz_diff_20'] = df.mz.diff(-20)

In [ ]:
df['mz_diff_31_cummin']= df['mz_diff_31'].cummin()
df['mz_diff_20_cummin']= df['mz_diff_20'].cummin()

In [ ]:
df[(df.mz_diff_20 <= df.mz_diff_20_cummin)].to_clipboard()

In [ ]:
df['mean_mz_diff']  = df.mz_diff.rolling(window = 31, center=True, win_type ='cosine' ).mean() # note win_type =should be tukey

In [ ]:
df['mz_std']  = df.mz_diff.rolling(window = 31, center=True ).std()

In [ ]:
df.to_clipboard()

In [ ]:

df['lenset'] = df.rolling(31)['scan_id_f'].apply(lambda s:len(set(s)))

In [ ]:
def ucounter(scanids_start,scanids_end, window_size):
    sid_cts = np.zeros(window_size+1, dtype=np.int64)
    uct = 0
    for scanid_start,scanid_end in zip(scanids_start,scanids_end):
        sid_cts[scanid_start] +=1
        if sid_cts[scanid_start] ==1:
            uct += 1
        sid_cts[int(scanid_end)] -=1
        if sid_cts[int(scanid_end)] ==0:
            uct -= 1
        yield uct
        


In [ ]:
df.scan_id_f.unique().size

In [ ]:
%%timeit
df['ucounter'] = list(ucounter(df.scan_id_f.to_numpy(),df.scan_id_f.shift(31).fillna(31).to_numpy(), 31))

In [ ]:

df['uniq=_size'] = df.rolling(31)['scan_id_f'].apply(lambda s:np.unique(s).size)

In [ ]:
df.to_clipboard()

In [ ]:
nunique_scans.max()

In [ ]:
df['mean_diff'] = df['mz'].diff(-1).rolling(56).mean()

In [ ]:
df.to_clipboard()

In [ ]:
df['mz_r'] = (df.mz / 10).round() *10

valids = df.loc[df['nunique_scans'] >= 56 * 0.9]


In [ ]:
valids['resolution'] = valids['mz'] / valids['mean_diff']

In [ ]:
valids.sort_values('mz', inplace = True)
valids['max_diff'] = valids['mean_diff'].cummax()


In [ ]:
valids.to_clipboard()

In [ ]:
valids.groupby('mz_r')['resolution'].min().to_frame().reset_index().plot(x='mz_r', y="resolution")

In [ ]:
def suggest_resolution_and_gradient(spectra_df):
    df = spectra_df.loc[spectra_df.precursor_id.isna()]
    scan_count = df.scan_id.unique().size

    df['scan_id_f'] = df['scan_id'].factorize()[0]
    df.sort_values(['mz','scan_id_f'], ascending=False, inplace=True)
    df['nunique_scans'] = df.rolling(60)['scan_id_f'].apply(lambda s:len(set(s)))
    max_unique_scans = df['nunique_scans'].max()
    df['mean_diff'] = df['mz'].diff(-1).rolling(max_unique_scans).mean()
    df['mz_r'] = (df.mz / 10).round() *10

    valids = df.loc[df['nunique_scans'] >= max_unique_scans * 0.9]
    valids.sort_values('mz', inplace = True)
    valids['max_diff'] = valids['mean_diff'].cummax()
    selected = valids.groupby('mz_r')['resolution'].min().to_frame().reset_index()
    # selected.plot(x='mz_r', y="resolution")
    res_at_loest_mass = selected['resolution'].iat[0]
    gradient = selected['resolution'].iat[-1] - selected['resolution'].iat[0 ] / selected['mz_r'].iat[-1] - selected['mz_r'].iat[0 ]
    return  res_at_loest_mass, gradient

In [ ]:
df['diff'] = df['mz'].diff(-1)
low,high = df['diff'].quantile([.20,.80])

In [ ]:
mzs = spectra_df.loc[spectra_df.precursor_id.isna()].mz.sort_values(ascending= False).to_frame()

In [ ]:
mzs['difs'] = mzs['mz'].diff(-1).replace(0,np.nan).fillna(method = 'bfill').fillna(method = 'ffill')

In [ ]:
low,high = mzs['difs'].quantile([.20,.80])

In [ ]:
mzs.loc[mzs['difs'].between(low,high), 'difs'] = np.nan

In [ ]:
mzs['cummin'] = mzs['difs'].cummin() 

In [ ]:
mzs.groupby('cummin')['mz'].last()

In [ ]:
absolute_tolerences.index.rename('da', inplace=True)
absolute_tolerences.reset_index(inplace=True)
absolute_tolerences['resolution'] = absolute_tolerences['mz'] / absolute_tolerences['da']
absolute_tolerences['mz'] = (absolute_tolerences['mz'] / 10).round() *10
absolute_tolerences['resolution'] = absolute_tolerences['resolution'].cummin()


In [ ]:
absolute_tolerences.to_clipboard()

In [ ]:
absolute_tolerences = absolute_tolerences.groupby('mz')['resolution'].min().to_frame()
absolute_tolerences.reset_index(inplace=True)

In [ ]:
absolute_tolerences.plot(x='mz',y='resolution')

In [ ]:
guess_selection_window (spectra_df)

In [ ]:
mean_less_std = agg_data.at['mean'] - agg_data.at['std']
if mean_less_std > 0:
    res =  mean_less_std / 2
else:
    res =  agg_data.at['mean'] /2

return res


In [ ]:
def drop_fuzzy(spectra_df):
    fraction_of_average_intensity = 0.1
    spectras_sum_inty =  spectra_df.loc[spectra_df.precursor_id.isna()].groupby('scan_id')['inty'].sum()
    sum_inty_mean = spectras_sum_inty.mean()
    spectras_sum_inty = spectras_sum_inty.to_dict()

    to_drop = []
    for scan_id in spectra_df.scan_id.drop_duplicates(): # this maintains order
        if spectras_sum_inty[scan_id] < sum_inty_mean * fraction_of_average_intensity: # one order 
            to_drop.append(scan_id)
        else:
            break
    
    return spectra_df.loc[~spectra_df.scan_id.isin(to_drop)]

In [ ]:
df = spectra_dfs[0] # first file, already teim range and mass range filtered
ms1_peaks = df.loc[df.precursor_id.isna()]

In [ ]:
df.stem.iloc[0]


In [ ]:
def bin_linear_alignment(masses, options):
    # TODO assert masses are ordered
    up_to = None
    for _, mass in masses.iteritems():
        if up_to is None:
            #up_to = mass + tolerance.getTinDA(mass) this is how its done in some places, but reuslt are not identical to below
            up_to = mass + mass / options["MSresolution"].tolerance 
        if mass <= up_to:
            yield up_to
        else:
            up_to = mass + mass / options["MSresolution"].tolerance
            yield up_to


# ms1 agg peaks similar to lx1...ms1_peaks_agg
# ms1 agg peaks as in lx2


In [ ]:
ms1_peaks.sort_values("mz", ascending=False)
scan_id_f = pd.factorize(ms1_peaks.scan_id)


In [ ]:
import pandas as pd
scan_id_f = pd.factorize(ms1_peaks.scan_id)

In [ ]:
bins1 = list(bin_linear_alignment(df.mz, options))
bins2 = list(bin_linear_alignment(df.groupby(bins1)["mz"].transform("mean"), options))
bins3 = list(bin_linear_alignment(df.groupby(bins2)["mz"].transform("mean"),options))

df['bin_mass'] = bins3 

In [ ]:
def ms1_peaks_agg(ms1_peaks, options):
    ms1_peaks.sort_values("mz", inplace=True)

    # binning is done 3 times in lx1, between each fadi filter is performed, we do it at the end intead
    bins1 = list(bin_linear_alignment(ms1_peaks.mz, options))
    bins2 = list(bin_linear_alignment(ms1_peaks.groupby(bins1)["mz"].transform("mean"), options))
    bins3 = list(bin_linear_alignment(ms1_peaks.groupby(bins2)["mz"].transform("mean"),options))

    ms1_peaks['bin_mass'] = bins3 

    # merge mutiple peaks from single scan
    g = ms1_peaks.groupby(["bin_mass", "scan_id"])
    ms1_peaks["scan_cumcount"] = g.cumcount()
    ms1_peaks["merged_mass"] = g["mz"].transform("mean")
    ms1_peaks['merged_inty'] = g['inty'].transform("mean") # NOTE merge is NOT weighted average 
    

    # aggregate results
    agg_df = (
        ms1_peaks.loc[ms1_peaks.scan_cumcount == 0] # use only the first of merged masses
        .assign(
            mass_intensity=lambda x: x.mz * x.merged_inty
        )  # for the weighted average intensity
        .groupby("bin_mass")
        .agg(
            {
                "merged_mass": ["mean", "count"],
                "merged_inty": ["mean", "sum"],
                "mass_intensity": "sum",
            }
        )
        .dropna()
    )
    agg_df.columns = ["_".join(col).strip() for col in agg_df.columns.values]

    # apply fadi filters, in lx1 its done between each bin  process
    fadi_denominator = ms1_peaks.scan_id.unique().shape[0]
    mask_ff = agg_df.merged_mass_count / fadi_denominator >= options['MSfilter']
    agg_df = agg_df[mask_ff]

    # NOTE intensity threshold is do in add_Sample... but lets do it here
    MSthreshold  = options["MSthreshold"]
    mask_inty = agg_df.merged_inty_mean > MSthreshold
    agg_df = agg_df[mask_inty]

    # for reference...weigted_mass shoud not be necesary
    agg_df["weigted_mass"] = agg_df.mass_intensity_sum / agg_df.merged_inty_sum
    # lx1 intensity is wrong because it uses the total number of scans, instead of the numebr of scans with a peak
    agg_df['lx1_bad_inty'] = agg_df.merged_inty_sum / fadi_denominator

    
    agg_df.rename(columns = {'merged_mass_mean':'mz', 'merged_inty_mean':'inty'}, inplace=True)
    return agg_df

In [ ]:
spectra_dfs[0].polarity.iat[0]

In [ ]:
res = {}
for df in spectra_dfs: # first file, already teim range and mass range filtered
    ms1_peaks = df.loc[df.precursor_id.isna()]
    agg_df = ms1_peaks_agg(ms1_peaks, options)
    agg_df['stem'] = df.stem.iloc[0]
    res[df.stem.iloc[0]] = agg_df

In [ ]:
#recalibrate
def recalibrate_mzs(mzs, cals):
    # from lx2_masterscan
    # lx1 takes all that are within tolerance and then uses highest intensity
    if not cals or mzs.empty:
        return mzs
    cal_matchs = [mzs.loc[mzs.sub(cal).abs().idxmin()] for cal in cals]
    
    cal_vals = [cal - cal_match for cal, cal_match in zip(cals, cal_matchs)]
    # prefilter
    if not any((v < 0.1 for v in cal_vals)):
        return mzs
    # find near tolerance
    cutoff = mzs.diff(-1).quantile(0.1)
    is_near = [v < cutoff for v in cal_vals]
    if not any(is_near):
        log.info("no valid calibration masses found")
        return mzs

    cal_matchs = [e for e, v in zip(cal_matchs, is_near) if v]
    cal_vals = [e for e, v in zip(cal_vals, is_near) if v]
    log.debug("recalibration info: {'\n'.join(zip(cal_matchs,cal_vals ))}")

    return mzs + np.interp(mzs, cal_matchs, cal_vals)

In [ ]:
#align ms1 
ms1_agg_peaks = pd.concat(res.values())

In [ ]:
ms1_agg_peaks

In [ ]:
# this one uses delta res for clustering
def bin_mkSurveyLinear(masses, options):
    # TODO assert masses are ordered
    minmass = masses.iloc[0]
    up_to = None
    for _, mass in masses.iteritems():
        if up_to is None:
            #up_to = mass + tolerance.getTinDA(mass) this is how its done in some places, but reuslt are not identical to below
            deltatol = options["MSresolution"].tolerance + (mass - minmass) * options['MSresolutionDelta']
            up_to = mass + (mass / deltatol)
        if mass <= up_to:
            yield up_to
        else:
            deltatol = options["MSresolution"].tolerance + (mass - minmass) * options['MSresolutionDelta']
            up_to = mass + (mass / deltatol)
            yield up_to

In [ ]:
# make the method here

In [ ]:
ms1_agg_peaks.sort_values('mz', inplace=True)

In [ ]:
# binning is done 3 times in lx1, between each fadi filter is performed, we do it at the end intead
bins1 = list(bin_mkSurveyLinear(ms1_agg_peaks.mz, options))
bins2 = list(bin_mkSurveyLinear(ms1_agg_peaks.groupby(bins1)["mz"].transform("mean"), options))
bins3 = list(bin_mkSurveyLinear(ms1_agg_peaks.groupby(bins2)["mz"].transform("mean"),options))

In [ ]:
ms1_agg_peaks['bins'] = bins3

In [ ]:
# check occupation spectracontainer.py masterscan.chekoccupation
# occupation is the % of peak intensities abvove "thrsld: "
threshold_denominator = ms1_agg_peaks.stem.unique().shape[0] # same as len(res)
threshold = options["MSminOccupation"]
bin_peak_count = ms1_agg_peaks.groupby('bins')['inty'].transform('count')
tf_mask = (bin_peak_count / threshold_denominator) >= threshold
ms1_agg_peaks['above_threshold'] = tf_mask

In [ ]:
ms1_agg_peaks['mass'] = ms1_agg_peaks.groupby('bins')['mz'].transform('mean')

In [ ]:
ms1_agg_peaks

In [ ]:
ms1_agg_peaks.pivot(index='mass', columns='stem', values=['inty','lx1_bad_inty'])

In [ ]:
# TODO collape_join_adjecent_clusters


# MS2 
----

In [ ]:
options["MSMSresolution"]
options["selectionWindow"]

#tolerance = TypeTolerance("Da", scan.options["selectionWindow"])
#window = scan.options["selectionWindow"] / 2

In [ ]:
df = spectra_dfs[1] # first file, already teim range and mass range filtered
ms2_peaks = pd.concat((df.loc[~df.precursor_id.isna()] for df in spectra_dfs))

ms2_peaks[ms2_peaks.scan_id == 'controllerType=0 controllerNumber=1 scan=41']  
*** NOTE masses are not exactly the same as in lx1, they are wrong base om raw ***  
see xcel file on desktop  

*** lx1 does the precursor binning all files, at the same time, it is not like with the ms1s ***

In [ ]:
def bin_linear_alignment_for_ms2(masses, telerance_da):
    # TODO assert masses are ordered
    up_to = None
    for _, mass in masses.iteritems():
        if up_to is None:
            #up_to = mass + tolerance.getTinDA(mass) this is how its done in some places, but reuslt are not identical to below
            up_to = mass + telerance_da
        if mass <= up_to:
            yield up_to
        else:
            up_to = mass + telerance_da
            yield up_to
    

In [ ]:
def grouped_precursors_df(ms2_peaks):
    ms2_peaks.sort_values(['precursor_mz','mz'], inplace=True)
    precursors_df = ms2_peaks[["stem", 'scan_id', 'precursor_mz']].drop_duplicates()# similar to unqie but return a series instead of an array
    # using "stem", 'scan_id' to replicate the numebr of instances for the averaging later
    
    bins1 = list(bin_linear_alignment_for_ms2(precursors_df.precursor_mz, options["selectionWindow"]))
    bins2 = list(bin_linear_alignment_for_ms2(precursors_df.groupby(bins1)['precursor_mz'].transform('mean'), options["selectionWindow"]))
    bins3 = list(bin_linear_alignment_for_ms2(precursors_df.groupby(bins2)['precursor_mz'].transform('mean'), options["selectionWindow"]))

    precursors_df['prec_bin'] = bins3

# no fadi filter for precursors  
# after groupinbg the precursors, the groups are split on the "sample" ie the file,  
# the each split of the group is merged in the __def mergeListsMsms__ method that uses linear alignment  
# thats why... prec_ms2_peaks = ms2_peaks[(ms2_peaks.precursor_mz == t) & (ms2_peaks.stem == '190321_Serum_Lipidextract_368723_01')]
    return precursors_df

In [ ]:
precursors_df = grouped_precursors_df(ms2_peaks)
precursors_bins = precursors_df.set_index('precursor_mz')['prec_bin'].to_dict()

In [ ]:
precursors_bins

in def linearAlignment the var clusterToMerge, it only takes the "sample" if its not in the cluster, so no duplicate, and only the first "sample"

for clutering ms2 precursors, it does the binning with linearAlignment... see above, its not the unique precusrsor, its each precursor in each file,  but it does the binning on both at the same time, in contrast to how its done for ms1 binning, where first the binning is done for one file and then for the other.

then it does the merge it on a per file (AKA "sample")  basis, merge means doing a linera alignmebt on all the peaks in the bin per sample


In [ ]:
ms2_peaks['prec_bin'] = ms2_peaks.precursor_mz.map(precursors_bins)


In [ ]:
# this one uses delta res for clustering
def bin_mkSurveyLinear_for_ms2(masses, options): # copied from above
    # TODO assert masses are ordered
    minmass = masses.iloc[0]
    up_to = None
    for _, mass in masses.iteritems():
        if up_to is None:
            #up_to = mass + tolerance.getTinDA(mass) this is how its done in some places, but reuslt are not identical to below
            deltatol = options["MSMSresolution"].tolerance + (mass - minmass) * options['MSMSresolutionDelta']
            up_to = mass + (mass / deltatol)
        if mass <= up_to:
            yield up_to
        else:
            deltatol = options["MSMSresolution"].tolerance + (mass - minmass) * options['MSMSresolutionDelta']
            up_to = mass + (mass / deltatol)
            yield up_to

In [ ]:
def ms2_peaks_group_generator(grouped_prec, options):

    for idx, prec_ms2_peaks in  grouped_prec:
        #intensityWeightedAvg
        prec_ms2_peaks.sort_values('mz', inplace=True)
        bins1 = list(bin_mkSurveyLinear_for_ms2(prec_ms2_peaks.mz, options))
        bins1_weighted_average = (prec_ms2_peaks.mz * prec_ms2_peaks.inty).groupby(bins1).transform('sum') / prec_ms2_peaks.inty.groupby(bins1).transform('sum')
        bins2 = list(bin_mkSurveyLinear_for_ms2(bins1_weighted_average, options))
        bins2_weighted_average = (prec_ms2_peaks.mz * prec_ms2_peaks.inty).groupby(bins2).transform('sum') / prec_ms2_peaks.inty.groupby(bins2).transform('sum')
        bins3 = list(bin_mkSurveyLinear_for_ms2(bins2_weighted_average, options))
        weighted_mass = (prec_ms2_peaks.mz * prec_ms2_peaks.inty).groupby(bins3).transform('sum') / prec_ms2_peaks.inty.groupby(bins3).transform('sum')

        prec_ms2_peaks['bins'] = bins3
        prec_ms2_peaks['weighted_mass'] = weighted_mass

        fadi_denominator = prec_ms2_peaks.scan_id.unique().shape[0]
        ff_mask = prec_ms2_peaks.groupby("bins")['bins'].transform('count') / fadi_denominator>= options['MSMSfilter']
        mof_mask = prec_ms2_peaks.groupby("bins")['bins'].transform('count') / fadi_denominator>= options['MSMSminOccupation']

        tf_mask = prec_ms2_peaks.inty > options["MSMSthreshold"]
        
        # it uses merge sum intensity for getting the averrage intensity...
        agg_prec_ms2_peaks = prec_ms2_peaks[ff_mask & tf_mask & mof_mask].groupby('bins').agg({"weighted_mass":['mean', 'count'],"inty":"mean"})
        agg_prec_ms2_peaks['precursor_mz'] = prec_ms2_peaks.precursor_mz.mean().round(6) 
        # there is minor differrence in mean between different files, and the same precursor bin, to avoid it we round
        
        agg_prec_ms2_peaks['stem'] = idx[0]

        agg_prec_ms2_peaks.columns = ["_".join(col).strip() for col in agg_prec_ms2_peaks.columns.values]
        names = {'weighted_mass_mean':'mz',
         'weighted_mass_count':'count', 
         "inty_mean":'inty',
         'precursor_mz_':'precursor_mz',
         'stem_':'stem'}
        agg_prec_ms2_peaks.rename(columns = names, inplace=True)

        yield agg_prec_ms2_peaks



In [ ]:
ms2_peaks.prec_bin.unique().size

In [ ]:
grouped_prec = ms2_peaks.groupby(['stem','prec_bin'])
ms2_agg_peaks = pd.concat(ms2_peaks_group_generator(grouped_prec, options))

In [ ]:
# collape_join_adjecent_clusters_msms(cluster)

In [ ]:
# associate ms2 to ms1 scans... 
ms1_masses = ms1_agg_peaks.mass.drop_duplicates()
precur_masses = ms2_agg_peaks.precursor_mz.drop_duplicates()
tol = options["selectionWindow"] / 2
tmp = pd.merge_asof(ms1_masses, precur_masses.astype(ms1_masses.dtype), left_on='mass', right_on='precursor_mz', direction ='nearest' , tolerance=tol)

In [ ]:
ms1_agg_peaks

# build the masterscan

In [ ]:
from lx.spectraContainer import MasterScan
from LX2_masterscan import se_factory, ms2entry_factory

In [ ]:
def mass_inty_generator_ms1_agg(ms1_agg_peaks):
    for mass, gdf in ms1_agg_peaks.groupby('mass'):
        dictIntensity = gdf.set_index('stem')['lx1_bad_inty'].to_dict()
        dictIntensity_lx2 = gdf.set_index('stem')['inty'].to_dict()
        dictIntensity.update({ f'{k}_lx2':v for k,v in dictIntensity_lx2.items()})
        yield (mass, dictIntensity, 1)

In [ ]:
samples = samples[:2]

In [ ]:
samples.extend([f'{k}_lx2' for k in samples]) #because we add both results

In [ ]:
samples

In [ ]:
listSurveyEntry = [
        se_factory(msmass, dictIntensity, samples, polarity)
        for msmass, dictIntensity, polarity in mass_inty_generator_ms1_agg(
            ms1_agg_peaks
        )
    ]

In [ ]:
def MSMSEntry_list_generator(gdf):
    for mz,  precur_df in gdf.groupby('mz'):
        dictIntensity = precur_df.set_index('stem')['inty'].to_dict()
        dictIntensity.update({ f'{k}_lx2':v for k,v in dictIntensity.items()}) #TODO actually get other values
        yield (mz, dictIntensity, samples, 1)

In [ ]:
def MS2_dict_generator(ms2_agg_peaks):
    for precursor_mz, gdf in ms2_agg_peaks.groupby('precursor_mz'):
        MSMSEntry_list = [ms2entry_factory(*args) for args in MSMSEntry_list_generator(gdf)]
        yield (precursor_mz, MSMSEntry_list)

In [ ]:
MS2_dict = dict(MS2_dict_generator(ms2_agg_peaks))

In [ ]:
MS2_dict_keys =pd.Series(list(MS2_dict.keys()), name = 'MS2_precurs')

In [ ]:
MS2_dict_keys.sort_values(inplace=True)

In [ ]:
MS1_precurmass =pd.Series([se.precurmass for se in listSurveyEntry], name = 'MS1_precurmass')

In [ ]:
MS1_precurmass.sort_values(inplace=True)

In [ ]:
# associate ms2 to ms1 scans
tol = options["selectionWindow"] / 2
precur_map_df = pd.merge_asof(MS1_precurmass, MS2_dict_keys, left_on='MS1_precurmass' , right_on='MS2_precurs',direction ='nearest' , tolerance=tol)
# tmp = pd.merge_asof(ms1_masses, ms1_masses.dtype, left_on='mass', right_on='precursor_mz', direction ='nearest' , tolerance=tol)

In [ ]:
precur_dict = precur_map_df.set_index(MS1_precurmass)['MS2_precurs'].to_dict()

In [ ]:
len(precur_dict)

In [ ]:
# ms1_prec_dict = ms1_agg_peaks.set_index('mass')['precursor_mz'].to_dict()
for se in listSurveyEntry:
    precursor = precur_dict[se.precurmass]
    print(len(MS2_dict.get(precursor,[])))
    

In [ ]:

# add data to masterscan
scan = MasterScan(options)
scan.listSurveyEntry = listSurveyEntry
scan.listSurveyEntry[0].massWindow = 0.01  # to avoid bug
scan.sampleOccThr["MS"] = [(0.0, [])]  # to avoid bug at def checkOccupation
scan.sampleOccThr["MSMS"] = [(0.0, [])]

# for printing we need
# samples.extend([f'{k}_lx2' for k in samples])
scan.listSamples = samples


In [ ]:
import pickle
with open('tmp_lx1_and_lx2.sc','wb') as f: pickle.dump(scan, f)

# recalibrate
import pickle

In [ ]:
import pickle
with open('ms1_dfs.pkl', 'rb') as f: ms1_dfs = pickle.load(f)
with open('options.pkl', 'rb') as f: options = pickle.load(f)

In [ ]:
#recalibrate
def recalibrate_mzs(mzs, cals):
    # from lx2_masterscan
    # lx1 takes all that are within tolerance and then uses highest intensity
    if not cals or mzs.empty:
        return mzs
    cal_matchs = [mzs.loc[mzs.sub(cal).abs().idxmin()] for cal in cals]

    cal_vals = [cal - cal_match for cal, cal_match in zip(cals, cal_matchs)]
    # prefilter
    if not any((v < 0.1 for v in cal_vals)):
        return mzs
    # find near tolerance
    cutoff = mzs.diff(-1).quantile(0.1)
    is_near = [v < cutoff for v in cal_vals]
    if not any(is_near):
        log.info("no valid calibration masses found")
        return mzs

    cal_matchs = [e for e, v in zip(cal_matchs, is_near) if v]
    cal_vals = [e for e, v in zip(cal_vals, is_near) if v]
    log.debug("recalibration info: {'\n'.join(zip(cal_matchs,cal_vals ))}")

    return mzs + np.interp(mzs, cal_matchs, cal_vals)

In [ ]:
res = []
for cal_mass in options['MScalibration']:
    tol = cal_mass / options["MSresolution"].tolerance
    #find close enough most intense
    reference_mass = ms1_df[ms1_df.mz.between(cal_mass-tol , cal_mass+tol)].sort_values('inty', ascending=False).mz.iat[0]
    change_val = cal_mass - reference_mass
    res.append((reference_mass, change_val))

cal_matchs, cal_vals = zip(*res)

# perf testing

In [ ]:
import pickle

In [ ]:
# with open('stitched.sc','wb') as f: pickle.dump(stitched, 

In [ ]:
with open('stitched.sc','rb') as f: stitched  = pickle.load(f) 

In [ ]:
stitched.dump('stitched-dump.csv')

In [ ]:
t = stitched.listSurveyEntry[0]
t.dictIntensity

In [3]:
import pickle
with open('tmp.pkl','rb') as f: resultSC = pickle.load(f)

In [ ]:
ms2_peaks[['precursor_mz', 'prec_bin']].drop_duplicates()['prec_bin'].value_counts()

In [ ]:
import pandas as pd


In [1]:
print('hello')

hello


In [5]:
from itertools import groupby

In [7]:
resultSC[0].listPrecurmassSF


[C20 H36 O7 N1 P1 D5 ]

In [29]:
merged_resultSC = []
for key,group in groupby(resultSC, lambda x: str(x.listPrecurmassSF)):
    base = next(group)
    for entry in group:
        for k,v in base.dictIntensity.items():
            base.dictIntensity[k] = max(v, entry.dictIntensity[k])
    merged_resultSC.append(base)
    

In [30]:
merged_resultSC

[/-/ . 443.2942 |           0            0        15317        18678        29218        27153        38954        26335        33533        40797            0        23153   QP: -1.0000 Mean: -1.0000 Median: -1.0000 V(X): -1.0000 E(X): -1.0000 [LPEd5_pr:443:MS1-],
 /-/ . 457.3093 |           0            0            0           94            0            0           78            0            0            0            0            0   QP: -1.0000 Mean: -1.0000 Median: -1.0000 V(X): -1.0000 E(X): -1.0000 [LPEd5_pr:457:MS1-],
 /-/ . 471.3254 |           0            0        38691        47712        71485        69443        87432        61811        75618       113022        33924        56082   QP: -1.0000 Mean: -1.0000 Median: -1.0000 V(X): -1.0000 E(X): -1.0000 [LPEd5_pr:471:MS1-],
 /-/ . 474.2870 |        1317            0        13925        18189        32972        32358        43318        28923        33385        50447         7505        14004   QP: -1.0000 Mean: -1.0000 M

# mfqls

In [54]:

import pickle
with open('mfqlObj.pkl','rb') as f: mfqlObj = pickle.load( f)

In [55]:
elementSequence = mfqlObj.dictDefinitionTable['Cerd7']['Cerd7_pr'].elementSequence

In [56]:
len(mfqlObj.sc.listSurveyEntry)

16566

In [57]:
options = mfqlObj.options

In [58]:
from lx.mfql.chemsc import calcSFbyMass

In [59]:
# check scope
#massrange 
calcSF_res = [calcSFbyMass(se.precurmass,elementSequence,options["MStolerance"]) for se in mfqlObj.sc.listSurveyEntry]
calcSF_res1 = [c for c in calcSF_res if c]

In [60]:
calcSF_res1[0][0].getWeight()
#se.listMark.append(mark)

587.5385864556

In [61]:
mfqlObj.dictSuchThat

{'Cerd7': <lx.mfql.runtimeStatic.TypeBooleanTerm at 0x241d8641608>,
 'DGd5': <lx.mfql.runtimeStatic.TypeBooleanTerm at 0x241dcb35f88>}

In [62]:
vars = mfqlObj.result.dictQuery['Cerd7'].sc[0].listVariables


In [63]:
vars

[{'Cerd7_pr': Cerd7_pr:587:MS1-}]

In [64]:
# do the isotopic correction before the suchthat
mfqlObj.dictSuchThat['Cerd7'].evaluate(scane=None,
mode="sc",
vars=vars,
queryName=mfqlObj.queryName,)

[[{'Cerd7_pr': Cerd7_pr:587:MS1-}]]